In [1]:
import pandas as pd

data = pd.read_csv("../input/updated-netflix-stock-price-all-time/netflix.csv")

In [2]:
data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2002-05-23,1.242857,1.145714,1.156429,1.196429,104790000.0,1.196429
1,2002-05-24,1.225000,1.197143,1.214286,1.210000,11104800.0,1.210000
2,2002-05-28,1.232143,1.157143,1.213571,1.157143,6609400.0,1.157143
3,2002-05-29,1.164286,1.085714,1.164286,1.103571,6757800.0,1.103571
4,2002-05-30,1.107857,1.071429,1.107857,1.071429,10154200.0,1.071429


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4881 entries, 0 to 4880
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       4881 non-null   object 
 1   High       4881 non-null   float64
 2   Low        4881 non-null   float64
 3   Open       4881 non-null   float64
 4   Close      4881 non-null   float64
 5   Volume     4881 non-null   float64
 6   Adj Close  4881 non-null   float64
dtypes: float64(6), object(1)
memory usage: 267.1+ KB


In [4]:
print(data.Close)
print(data.columns)
date = data["Date"]
date_list = (date.values)
Close = data.Close.values
print(date_list)
print(data["Date"])

0         1.196429
1         1.210000
2         1.157143
3         1.103571
4         1.071429
           ...    
4876    634.809998
4877    639.099976
4878    631.849976
4879    632.659973
4880    627.039978
Name: Close, Length: 4881, dtype: float64
Index(['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'], dtype='object')
['2002-05-23' '2002-05-24' '2002-05-28' ... '2021-10-07' '2021-10-08'
 '2021-10-11']
0       2002-05-23
1       2002-05-24
2       2002-05-28
3       2002-05-29
4       2002-05-30
           ...    
4876    2021-10-05
4877    2021-10-06
4878    2021-10-07
4879    2021-10-08
4880    2021-10-11
Name: Date, Length: 4881, dtype: object


將全部資料每六十天拆成一個區塊
目標使用前六十天的股價作為預測第六十一天的股價的training資料

In [5]:
print(pd.date_range(start = "2002-05-23",end = "2021-10-11", freq="62d"))
print(len(pd.date_range(start = "2002-05-23",end = "2021-10-11", freq="62d")))

print(len(date.values))
print(4881/61)
#test = data["Date"] <= "2016-01-05"
#data[test]


DatetimeIndex(['2002-05-23', '2002-07-24', '2002-09-24', '2002-11-25',
               '2003-01-26', '2003-03-29', '2003-05-30', '2003-07-31',
               '2003-10-01', '2003-12-02',
               ...
               '2020-03-19', '2020-05-20', '2020-07-21', '2020-09-21',
               '2020-11-22', '2021-01-23', '2021-03-26', '2021-05-27',
               '2021-07-28', '2021-09-28'],
              dtype='datetime64[ns]', length=115, freq='62D')
115
4881
80.01639344262296


In [6]:
data.iloc[0].values #抓特定一筆資料
data[data["Date"]<"2012-05-23"]
print(len(data))
#data.loc["2002-05-23"]

4881


In [7]:
#把closed 的價格抓起來並變成np.array
high_closed_price  = data.iloc[: , [5,2]].values #iloc.[:,[2,5]]抓整行，且第2,5行
#high_closed_price[:,0]=data["Close"].value

print(data["Close"].values)


[  1.19642901   1.21000004   1.157143   ... 631.84997559 632.65997314
 627.03997803]


In [8]:
#標準化training
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
high_closed_price_scaled = sc.fit_transform(high_closed_price)
print(high_closed_price_scaled)
close_scaled = high_closed_price_scaled
print(close_scaled)
print(close_scaled.shape)

[[0.32341447 0.00126767]
 [0.03348267 0.00134924]
 [0.01957055 0.0012858 ]
 ...
 [0.01012384 0.99934978]
 [0.00923936 1.        ]
 [0.00797083 0.99352915]]
[[0.32341447 0.00126767]
 [0.03348267 0.00134924]
 [0.01957055 0.0012858 ]
 ...
 [0.01012384 0.99934978]
 [0.00923936 1.        ]
 [0.00797083 0.99352915]]
(4881, 2)


In [9]:
#附上time,stamp
import numpy as np
x = [] #前60天的價格
y = [] #第61天的價格
for i in range(60, len(data)):
    x.append(close_scaled[i-60:i, 0])
    y.append(close_scaled[i, 0])
x, y = np.array(x), np.array(y)
print(x.shape, x)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))
print(x.shape,x)

(4821, 60) [[0.32341447 0.03348267 0.01957055 ... 0.01164181 0.00921553 0.00477457]
 [0.03348267 0.01957055 0.02002981 ... 0.00921553 0.00477457 0.00553712]
 [0.01957055 0.02002981 0.0305408  ... 0.00477457 0.00553712 0.00697556]
 ...
 [0.01353611 0.0167992  0.00976856 ... 0.01457718 0.02862237 0.01329131]
 [0.0167992  0.00976856 0.0111417  ... 0.02862237 0.01329131 0.01012384]
 [0.00976856 0.0111417  0.02056396 ... 0.01329131 0.01012384 0.00923936]]
(4821, 60, 1) [[[0.32341447]
  [0.03348267]
  [0.01957055]
  ...
  [0.01164181]
  [0.00921553]
  [0.00477457]]

 [[0.03348267]
  [0.01957055]
  [0.02002981]
  ...
  [0.00921553]
  [0.00477457]
  [0.00553712]]

 [[0.01957055]
  [0.02002981]
  [0.0305408 ]
  ...
  [0.00477457]
  [0.00553712]
  [0.00697556]]

 ...

 [[0.01353611]
  [0.0167992 ]
  [0.00976856]
  ...
  [0.01457718]
  [0.02862237]
  [0.01329131]]

 [[0.0167992 ]
  [0.00976856]
  [0.0111417 ]
  ...
  [0.02862237]
  [0.01329131]
  [0.01012384]]

 [[0.00976856]
  [0.0111417 ]
  [0.

# Train_Test_split

In [10]:
#train_test_split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)


In [11]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [12]:
print(x_train.shape)
print(y_train.shape)
#有3856個 內含六十個含有一個陣列的陣列

(3856, 60, 1)
(3856,)


In [13]:
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1))) #units 把每個日期的價格變成50維
model.add(Dropout(0.5))

# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.5))

# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.5))

# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.5))

model.add(Dense(units=1))
model.summary()

2022-03-30 05:30:34.718766: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 05:30:34.805362: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 05:30:34.806053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-30 05:30:34.807511: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 60, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

# Define when to save

In [14]:
from tensorflow.keras import callbacks #這邊是keras自己的東西，自動儲存model
filepath = '/kaggle/working/best1.hdf5'
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='min', verbose=1)
#monitor:取失誤最小的
callbacks_list = [checkpoint]

In [15]:
BATCH_SIZE=64

In [16]:
import tensorflow #optomizer 優化
from tensorflow.keras import metrics
#這邊是adam也很好用
# opt = tensorflow.compat.v1.train.AdamOptimizer(
#     learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False,
#     name='Adam'
# )
#這邊是SGD一種optomizer 
#opt = tensorflow.keras.optimizers.SGD( #最好用的之一
#    learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD" #learning rate調整學習速率，自己嘗試
#)
#model.compile(optimizer=opt, loss='mean_squared_error') 

In [17]:
#optimizer
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [18]:
#from tensorflow.keras import metrics, optimizers
#import tensorflow
#opt = tensorflow.compat.v1.train.AdamOptimizer(
#    learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False,
#    name='Adam'
#) #learning_rate調整反調整的rate
#model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', metrics.AUC()])

In [19]:
model.fit(x_train, y_train, epochs = 100, batch_size = 32)

2022-03-30 05:30:38.648657: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-03-30 05:30:43.951334: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


121/121 [==============================] - 8s 13ms/step - loss: 0.0027
Epoch 2/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0025
Epoch 3/100
121/121 [==============================] - 2s 14ms/step - loss: 0.0025
Epoch 4/100
121/121 [==============================] - 2s 17ms/step - loss: 0.0025
Epoch 5/100
121/121 [==============================] - 2s 14ms/step - loss: 0.0024
Epoch 6/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0024
Epoch 7/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0024
Epoch 8/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0023
Epoch 9/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0024
Epoch 10/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0023
Epoch 11/100
121/121 [==============================] - 2s 14ms/step - loss: 0.0023
Epoch 12/100
121/121 [==============================] - 2s 13ms/step - loss: 0.0023
Epoch 13/100
